## 结合工具规划逻辑

包括 ReAct、ReWoo、Plan 等复杂的工具规划推理也可以通过 FlowAgent 实现。

## ReAct

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_weather(city: str):
    """我可以查询城市的天气情况"""
    yield f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    yield '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(planner=ChatQwen(), tools=[get_weather, booking])
flow("如果不下雨，就帮我预订广州的羽毛球场")
flow.agents[0].memory

[AGENT] STEP 1 >>> Node 1: planner
**思考**

首先，我们需要知道广州的天气情况来判断是否下雨。这一步需要调用 `get_weather` 工具来获取信息。如果天气情况允许（即没有下雨），那么接下来的任务就是预订羽毛球场。这一步需要调用 `booking` 工具。因此，首先进行的步骤应该是查询广州的天气。

**规划**

首先，我将使用 `get_weather` 工具来查询广州的天气。

<self_solve>
无需自我解决部分，因为可以直接通过工具查询天气。
</self_solve>

**行动**

<sub_task>
{
    "name": "get_weather",
    "arguments": {
        "city": "广州"
    }
}
</sub_task>
广州天气不错，一直是大晴天


**观察** 上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] STEP 2 >>> Node 1: planner
**思考**

既然广州的天气情况是大晴天，也就是没有下雨，那么接下来的任务就是预订羽毛球场了。这个任务可以通过调用 `booking` 工具来实现。

**规划**

直接通过 `booking` 工具来预订广州的羽毛球场。

**行动**

<sub_task>
{
    "name": "booking",
    "arguments": {
        "request": "在广州预订一个羽毛球场"
    }
}
</sub_task>
我已经帮你预订好了，祝你玩得愉快


**观察** 上面的行动结果为:
我已经帮你预订好了，祝你玩得愉快
[AGENT] STEP 3 >>> Node 1: planner
**思考**

既然已经成功预订了广州的羽毛球场，并且得到了确认回复“我已经帮你预订好了，祝你玩得愉快”，这意味着我们的任务已经完成。

**规划**

当前情况下，我们已经完成了所有必要的步骤——检查了天气情况并且成功预订了羽毛球场。因此，不需要再进行任何进一步的行动或子任务。

**行动**

在这种情况下，我们不需要执行任何新的子任务。

**观察**

既然没有新的子任务，也没有新的观察结果需要报告。

**思考**

现在我们可

[{'role': 'system',
  'content': '尽你所能完成任务，必要时选择一个工具来解决问题: ** get_weather,booking\n\n**工具的详细描述如下:**\n{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}\n{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}\n\n**你要解决的问题是: ** 如果不下雨，就帮我预订广州的羽毛球场\n\n\n\n**思考**\n\n首先，我们需要知道广州的天气情况来判断是否下雨。这一步需要调用 `get_weather` 工具来获取信息。如果天气情况允许（即没有下雨），那么接下来的任务就是预订羽毛球场。这一步需要调用 `booking` 工具。因此，首先进行的步骤应该是查询广州的天气。\n\n**规划**\n\n首先，我将使用 `get_weather` 工具来查询广州的天气。\n\n<self_solve>\n无需自我解决部分，因为可以直接通过工具查询天气。\n</self_solve>\n\n**行动**\n\n<sub_task>\n{\n    "name": "get_weather",\n    "arguments": {\n        "city": "广州"\n    }\n}\n</sub_task>\n\n**观察** 上面的行动结果为:\n广州天气不错，一直是大晴天\n\n**思考**\n\n既然广州的天气情况是大晴天，也就是没有下雨，那

In [6]:
flow.agents[0].consumer_dict.keys()

dict_keys(['last_output', 'resources', 'completed_work'])

In [7]:
flow.max_steps

20

## ReWoo